# 모두를 위한 딥러닝 시즌2 - TensorFlow

## LAB06-1 - Softmax Classifier

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777)

2.5.0


In [2]:
#data

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.asarray(x_data, dtype =np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [4]:
#dataset 선언
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(2)

#number of classes
nb_classes = 3
print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [7]:
#weight와 bias 세팅하기
W= tf.Variable(tf.random.normal((4,nb_classes)),name ='weight')#4개의 특징값과 3개의 class값
b= tf.Variable(tf.random.normal((nb_classes,)),name='bias')
variables = [W,b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 1.7266569 ,  0.93108284, -0.43655586],
       [-1.2439743 , -0.45300987, -0.51410425],
       [ 1.2241375 ,  2.1267812 , -1.6276842 ],
       [-0.11408016, -0.08377677,  0.12709002]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.27697366, -2.3162541 ,  1.486129  ], dtype=float32)>


In [8]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X,W)+b)

print(hypothesis(x_data))

tf.Tensor(
[[5.1594371e-01 3.7466007e-01 1.0939622e-01]
 [5.1771075e-01 4.8226342e-01 2.5805663e-05]
 [6.9122940e-01 3.0876419e-01 6.4158858e-06]
 [4.4173509e-01 5.5826491e-01 1.9998740e-09]
 [6.3159363e-04 9.9936825e-01 1.5016340e-07]
 [3.1004906e-02 9.6899486e-01 2.4400759e-07]
 [5.4803921e-04 9.9945194e-01 4.6149586e-09]
 [9.7796328e-05 9.9990225e-01 1.2556407e-10]], shape=(8, 3), dtype=float32)


In [9]:
#softmax one hot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype =np.float32)

print(hypothesis(sample_db))
#첫번째 레이블이 가장 큰 값, 이게 선정됨.

tf.Tensor([[9.9697566e-01 3.0241790e-03 1.1561164e-07]], shape=(1, 3), dtype=float32)


### Cost function : cross entropy

In [10]:
def cost_fn(X,Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y*tf.math.log(logits),axis = 1)
    cost_mean= tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(5.2614083, shape=(), dtype=float32)


tf.constatant : 파이썬 리스트, 넘파이 어레이로 부터 상수텐서 생성 (당연하게 tf.Variable 과는 다르게 변하지 않음)

In [12]:
x=tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x*x

dy_dx = g.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [14]:
def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss, variables)
        
        return grads
    
print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.4280345 ,  0.30828202, -0.73631656],
       [-1.2808793 ,  1.7535259 , -0.4726467 ],
       [-0.8108007 ,  1.6721138 , -0.8613131 ],
       [-0.7852352 ,  1.6465507 , -0.8613155 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.02486265,  0.33645874, -0.3613214 ], dtype=float32)>]


In [15]:
def fit(X,Y, epochs = 2000, verbose = 100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
    
    for i in range(epochs):
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads, variables))
        if(i==0)|((i+1)%verbose ==0):
            print('loss at epoch %d: %f'%(i+1, cost_fn(X,Y).numpy()))
fit(x_data, y_data)

loss at epoch 1: 4.102696
loss at epoch 100: 0.646512
loss at epoch 200: 0.568314
loss at epoch 300: 0.515267
loss at epoch 400: 0.468391
loss at epoch 500: 0.423284
loss at epoch 600: 0.378449
loss at epoch 700: 0.333311
loss at epoch 800: 0.287950
loss at epoch 900: 0.244896
loss at epoch 1000: 0.220589
loss at epoch 1100: 0.210063
loss at epoch 1200: 0.200549
loss at epoch 1300: 0.191842
loss at epoch 1400: 0.183843
loss at epoch 1500: 0.176471
loss at epoch 1600: 0.169654
loss at epoch 1700: 0.163332
loss at epoch 1800: 0.157454
loss at epoch 1900: 0.151976
loss at epoch 2000: 0.146857


In [16]:
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a=hypothesis(sample_data)

print(a)
print(tf.argmax(a,1))

tf.Tensor([[0.0013511  0.06746063 0.9311883 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [17]:
b=hypothesis(x_data)
print(b)
print(tf.argmax(b,1))
print(tf.argmax(y_data,1))

tf.Tensor(
[[3.9563870e-06 1.3164548e-03 9.9867958e-01]
 [1.3511032e-03 6.7460634e-02 9.3118829e-01]
 [2.9748460e-08 1.4705150e-01 8.5294855e-01]
 [7.3429032e-07 8.6910737e-01 1.3089189e-01]
 [2.5018749e-01 7.3943979e-01 1.0372744e-02]
 [1.3216095e-01 8.6721015e-01 6.2882260e-04]
 [7.5919122e-01 2.4070939e-01 9.9491044e-05]
 [9.2029667e-01 7.9701357e-02 1.9764909e-06]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [20]:
class softmax_classifier(tf.keras.Model):
    def __init__(self,nb_classes):
        super(softmax_classifier, self).__init__()
        self.W = tf.Variable(tf.random.normal((4,nb_classes)),name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)),name ='bias')
        
    def softmax_regression(self,X):
        return tf.nn.softmax(tf.matmul(X,self.W)+self.b)
    
    def cost_fn(self, X,Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.math.log(logits), axis=1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
                
model = softmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 7.899121
Loss at epoch 500: 0.493023
Loss at epoch 1000: 0.268561
Loss at epoch 1500: 0.194537
Loss at epoch 2000: 0.159601
